In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, date_format
import sys
import os
from apis.airportdb import AirportDB

spark = SparkSession.builder.appName("EDA with PySpark").getOrCreate()

24/09/02 19:49:02 WARN Utils: Your hostname, victor-gabriel-pc resolves to a loopback address: 127.0.1.1; using 192.168.31.210 instead (on interface wlp2s0)
24/09/02 19:49:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/02 19:49:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
os.getcwd()

'/home/victor-gabriel/Documents/repos/picpay-ml-eng/notebook'

In [3]:
df = spark.read.csv("../data/airports-database.csv", header=True, inferSchema=True)

In [4]:
df.show(5)

+---+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+--------------------+
| id|year|month|day|dep_time|sched_dep_time|dep_delay|arr_time|sched_arr_time|arr_delay|carrier|flight|tailnum|origin|dest|air_time|distance|hour|minute|          time_hour|                name|
+---+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+--------------------+
|  0|2013|    1|  1|   517.0|           515|      2.0|   830.0|           819|     11.0|     UA|  1545| N14228|   EWR| IAH|   227.0|    1400|   5|    15|2013-01-01 05:00:00|United Air Lines ...|
|  1|2013|    1|  1|   533.0|           529|      4.0|   850.0|           830|     20.0|     UA|  1714| N24211|   LGA| IAH|   227.0|    1416|   5|    29|2013-01-01 05:00:00|United Air Lines ...|
|  2|2013|    1|  1|   54

In [5]:
#get size of the data
print((df.count(), len(df.columns)))

(336776, 21)


In [6]:
#get flight unique values 
df.select("flight").distinct().count()

3844

In [7]:
df[df["flight"] == 1959].show(5)

+----+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+--------------------+
|  id|year|month|day|dep_time|sched_dep_time|dep_delay|arr_time|sched_arr_time|arr_delay|carrier|flight|tailnum|origin|dest|air_time|distance|hour|minute|          time_hour|                name|
+----+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+--------------------+
| 110|2013|    1|  1|   804.0|           810|     -6.0|  1103.0|          1116|    -13.0|     DL|  1959| N947DL|   JFK| MCO|   147.0|     944|   8|    10|2013-01-01 08:00:00|Delta Air Lines Inc.|
| 991|2013|    1|  2|   800.0|           810|    -10.0|  1102.0|          1116|    -14.0|     DL|  1959| N995DL|   JFK| MCO|   143.0|     944|   8|    10|2013-01-02 08:00:00|Delta Air Lines Inc.|
|1935|2013|    1|  3

In [8]:
df.groupBy("flight").count().show(5)

+------+-----+
|flight|count|
+------+-----+
|   496|   10|
|  1959|  330|
|  4935|   32|
|   471|  129|
|  1580|   92|
+------+-----+
only showing top 5 rows



Temos 336776 registros de voos. <br/>
Temos 3844 registros único de voos, ou seja, 3844 voos que foram feitos mais de uma vez e com diferentes aeronaves, atrasos, número da cauda, etc. <br/>

In [9]:
#filter df dep_time = null and arr_time = 
df.filter(df["dep_time"].isNull() & df["arr_time"].isNull()).count() 

8255

8255 registros de voos foram cancelados. <br/>

In [10]:
df.select("dep_delay").describe().show()

+-------+------------------+
|summary|         dep_delay|
+-------+------------------+
|  count|            328521|
|   mean|12.639070257304708|
| stddev| 40.21006089212997|
|    min|             -43.0|
|    max|            1301.0|
+-------+------------------+



In [11]:
not_cancelled_flights = df.filter(df["dep_time"].isNotNull() | df["arr_time"].isNotNull())

In [12]:
not_cancelled_flights.groupBy("dest").count().orderBy("count", ascending=False).show(5)

+----+-----+
|dest|count|
+----+-----+
| ATL|16898|
| ORD|16642|
| LAX|16076|
| BOS|15049|
| MCO|13982|
+----+-----+
only showing top 5 rows



In [13]:
df.groupBy(["origin", "dest"]).count().orderBy("count", ascending=False).show(5)

+------+----+-----+
|origin|dest|count|
+------+----+-----+
|   JFK| LAX|11262|
|   LGA| ATL|10263|
|   LGA| ORD| 8857|
|   JFK| SFO| 8204|
|   LGA| CLT| 6168|
+------+----+-----+
only showing top 5 rows



Maior tempo médio de atraso

In [14]:
df.groupBy("dest") \
    .mean("dep_delay") \
    .withColumnRenamed("avg(dep_delay)", "avg_dep_delay") \
    .orderBy("avg_dep_delay", ascending=False) \
    .show(5)

+----+------------------+
|dest|     avg_dep_delay|
+----+------------------+
| CAE|35.570093457943926|
| TUL| 34.90635451505017|
| OKC|30.568807339449542|
| BHM| 29.69485294117647|
| TYS|28.493955094991364|
+----+------------------+
only showing top 5 rows



Dia da semana com maior número de voos

In [15]:
df.select(['time_hour']).show()
df_with_weekday = df.withColumn(
    "day_of_week",
    date_format(col("time_hour"), "EEEE")
)

+-------------------+
|          time_hour|
+-------------------+
|2013-01-01 05:00:00|
|2013-01-01 05:00:00|
|2013-01-01 05:00:00|
|2013-01-01 05:00:00|
|2013-01-01 06:00:00|
|2013-01-01 05:00:00|
|2013-01-01 06:00:00|
|2013-01-01 06:00:00|
|2013-01-01 06:00:00|
|2013-01-01 06:00:00|
|2013-01-01 06:00:00|
|2013-01-01 06:00:00|
|2013-01-01 06:00:00|
|2013-01-01 06:00:00|
|2013-01-01 06:00:00|
|2013-01-01 05:00:00|
|2013-01-01 06:00:00|
|2013-01-01 06:00:00|
|2013-01-01 06:00:00|
|2013-01-01 06:00:00|
+-------------------+
only showing top 20 rows



In [16]:
df_with_weekday.select(["time_hour", "day_of_week"]).show(5)

+-------------------+-----------+
|          time_hour|day_of_week|
+-------------------+-----------+
|2013-01-01 05:00:00|    Tuesday|
|2013-01-01 05:00:00|    Tuesday|
|2013-01-01 05:00:00|    Tuesday|
|2013-01-01 05:00:00|    Tuesday|
|2013-01-01 06:00:00|    Tuesday|
+-------------------+-----------+
only showing top 5 rows



In [17]:
df_with_weekday.groupBy("day_of_week").count().orderBy("count", ascending=False).show()

+-----------+-----+
|day_of_week|count|
+-----------+-----+
|     Monday|50690|
|    Tuesday|50422|
|     Friday|50308|
|   Thursday|50219|
|  Wednesday|50060|
|     Sunday|46357|
|   Saturday|38720|
+-----------+-----+



Data Split for Model Training

In [18]:
df_with_weekday.show(2)

+---+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+--------------------+-----------+
| id|year|month|day|dep_time|sched_dep_time|dep_delay|arr_time|sched_arr_time|arr_delay|carrier|flight|tailnum|origin|dest|air_time|distance|hour|minute|          time_hour|                name|day_of_week|
+---+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+--------------------+-----------+
|  0|2013|    1|  1|   517.0|           515|      2.0|   830.0|           819|     11.0|     UA|  1545| N14228|   EWR| IAH|   227.0|    1400|   5|    15|2013-01-01 05:00:00|United Air Lines ...|    Tuesday|
|  1|2013|    1|  1|   533.0|           529|      4.0|   850.0|           830|     20.0|     UA|  1714| N24211|   LGA| IAH|   227.0|    1416|   5|    29|2013-01-01 05:00:00

In [19]:
X_columns = ["distance", "dep_delay", "arr_delay"]
y_columns = "air_time"

In [20]:
X = df.select(X_columns)
y = df.select(y_columns)

In [21]:
y.show()

+--------+
|air_time|
+--------+
|   227.0|
|   227.0|
|   160.0|
|   183.0|
|   116.0|
|   150.0|
|   158.0|
|    53.0|
|   140.0|
|   138.0|
|   149.0|
|   158.0|
|   345.0|
|   361.0|
|   257.0|
|    44.0|
|   337.0|
|   152.0|
|   134.0|
|   147.0|
+--------+
only showing top 20 rows



In [33]:
import requests
weatherbit_key = "1f22b701b6904b66a169a5f52eddbed4"
latitude = 40.7128
longitude = -74.0060
start_date = '2023-01-01'
end_date = '2023-01-02' # Sempre adicione +1 dia em relação ao start_date
url = 'https://api.weatherbit.io/v2.0/history/daily'
params = {
'lat': latitude,
'lon': longitude,
'start_date': start_date,
'end_date': end_date,
'key': weatherbit_key,
}
headers = {
'Accept': 'application/json',
}
response = requests.get(url, params=params, headers=headers)
data = response.json()
data


{'city_id': '5128581',
 'city_name': 'New York City',
 'country_code': 'US',
 'data': [{'clouds': 37,
   'datetime': '2023-01-01',
   'dewpt': 5.7,
   'dhi': 27,
   'dni': 230,
   'ghi': 99,
   'max_dhi': 92,
   'max_dni': 773,
   'max_ghi': 427,
   'max_temp': 11.9,
   'max_temp_ts': 1672603200,
   'max_uv': 2.2,
   'max_wind_dir': 120,
   'max_wind_spd': 5,
   'max_wind_spd_ts': 1672588800,
   'min_temp': 7.3,
   'min_temp_ts': 1672624800,
   'precip': 0.4,
   'precip_gpm': 0.4,
   'pres': 1010,
   'revision_status': 'final',
   'rh': 77,
   'slp': 1013,
   'snow': 0,
   'snow_depth': None,
   'solar_rad': 97,
   't_dhi': 639,
   't_dni': 5510,
   't_ghi': 2378,
   't_solar_rad': 2317,
   'temp': 9.8,
   'ts': 1672549200,
   'wind_dir': 120,
   'wind_gust_spd': 8.2,
   'wind_spd': 3}],
 'lat': 40.7128,
 'lon': -74.006,
 'sources': ['997271-99999',
  '720553-99999',
  'imerg',
  'era5',
  'modis',
  'snodas'],
 'state_code': 'NY',
 'station_id': '720553-99999',
 'timezone': 'America/N

In [41]:
data['data'][0]['wind_spd']

3

In [24]:
origins = df.select('origin').distinct()
dests = df.select('dest').distinct()
dests = dests.withColumnRenamed('dest', 'origin')
unique_locations = origins.union(dests).distinct()
location_list = [row['origin'] for row in unique_locations.collect()]

In [27]:
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, parent_dir)

In [30]:
location_dict = AirportDB.get_airport_data(location_list)

In [46]:
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, FloatType

def get_location(airport_code):
    return location_dict.get(airport_code, None)

In [47]:
get_location_udf = udf(get_location,  ArrayType(FloatType()))

df_with_coordinates = df.withColumn("origin_coordinates", get_location_udf(df["origin"]))

df_with_coordinates.show(truncate=False)

+---+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+------------------------+-------------------+
|id |year|month|day|dep_time|sched_dep_time|dep_delay|arr_time|sched_arr_time|arr_delay|carrier|flight|tailnum|origin|dest|air_time|distance|hour|minute|time_hour          |name                    |coordinates        |
+---+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+------------------------+-------------------+
|0  |2013|1    |1  |517.0   |515           |2.0      |830.0   |819           |11.0     |UA     |1545  |N14228 |EWR   |IAH |227.0   |1400    |5   |15    |2013-01-01 05:00:00|United Air Lines Inc.   |[40.6925, -74.1687]|
|1  |2013|1    |1  |533.0   |529           |4.0      |850.0   |830           |20.0     |UA     |1714  |N24211 |LGA   |IAH |2